In [3]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
#Same as before

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [1]:
#TrainingArguments用于配置训练过程中的设置，如学习率、训练周期、批次大小等
from transformers import TrainingArguments

#指定模型训练的输出目录为test-trainer
#TrainingArguments会在该目录下存储文件模型和训练的输出，如checkpoint,log等
training_args = TrainingArguments("test-trainer")

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
#利用Trainer进行fine-tuning

"""
fine-tuning就是在一个已经预训练好的模型基础上，使用特定任务的数据进行再训练，让模型更好的适应你要做的任务；
fine-tuning过程中，主要更改的是模型的参数（数据如训练集、模型权重、偏置等），但并不仅仅只能更改它，还能改：
  超参数（learning rate、batch size、epochs、optimizers）；
  对training data和validation data进行不同的预处理（数据清洗、数据增强、tokenization）；
  模型架构（修改模型输出层——二分类变多分类、层数/隐藏单元数，添加dropout层）；
"""

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

<ipython-input-11-0790ca97a370>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
#用validation数据进行预测
predictions = trainer.predict(tokenized_datasets["validation"])
#打印出预测结果和标签的形状
print(predictions.predictions.shape, predictions.label_ids.shape)

上面这行代码运行输出为：(408,2) (408,)

  (408,2)表示408个样本，每个样本有2个类别；
  (408,)表示408个样本的标签（0或1）

In [ ]:
#找到每个样本预测结果中 概率最大的类别
#preds是一个数组，里面包含了模型对所有样本的预测类别
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate

#加载了MRPC任务的评估标准（准确率、F1分数等指标）
metric = evaluate.load("glue", "mrpc")
#计算模型在MRPC任务上的评估指标
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8578431372549019, 'f1': 0.8996539792387542}

In [ ]:
def compute_metrics(eval_preds):
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
#每训练完一个epoch后，在validation data上评估一次性能
training_args = TrainingArguments("test,trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_label=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenzier,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()